In this tutorial, we will use the adaptive step solver kernel to calculate the Smaller Aligment Index (SALI) for the Hénon-Heiles system

In [1]:
# Import libraries
import numpy as np
from numba import cuda, float64
import math
import time
import sys
import os
from numpy.random import default_rng
try:
    from chaoticus import (create_solver_kernel_adaptive_variational,
                                 calc_SALI)
except ImportError:
    print("Error: Could not import functions from 'chaoticus.py'.")
    print("Make sure the file exists and is in the correct path.")
    sys.exit(1)

The Hénon-Heiles system Hamiltonian is defined as:

$$
    \mathcal{H}(x, y, p_x, p_y) = \dfrac{p_x^{2}}{2} + \dfrac{p_y^{2}}{2} + \mathcal{V}(x, y)
$$

where the potential energy is given by:

$$
    \mathcal{V}(x, y) = \dfrac{1}{2} (x^{2} + y^{2}) + x^{2}y - \dfrac{1}{3}y^{3}
$$

Then, Hamilton's equations of motion result to be the following:

$$
    \begin{cases}
        \dot{x} = p_x \\
        \dot{y} = p_y \\
        \dot{p_x} = -x - 2xy \\
        \dot{p_y} = -y - x^{2} + y^{2}
    \end{cases}
$$

and as SALI is a chaos indicator that employes devition vectors, the variational equations for the Hénon-Heiles system are:

$$
    \begin{cases}
        \dot{\delta x} = \delta p_x \\
        \dot{\delta y} = \delta p_y \\
        \dot{\delta p_x} = - (1 + 2y) \delta x - 2x \delta y \\
        \dot{\delta p_y} = - 2x \delta x + (2y - 1) \delta y
    \end{cases}
$$

In the case of SALI, only two deviation vectors are needed for it's computation. Then, defining $\boldsymbol{v}_1$ and $\boldsymbol{v}_2$ as two deviation vectors whose initial conditions where linearly independent, SALI is given by:

$$
    \text{SALI} = \text{min}(|\boldsymbol{v}_1 + \boldsymbol{v}_2|, |\boldsymbol{v}_1 - \boldsymbol{v}_2|)
$$
where it is a good practice to use normalized deviation vectors to avoid very large or small values that can lead to numerical instabilities.



In [2]:
"""
Define the Hamiltonian function
"""
def H_henon_heiles_cpu(x, y, px, py):
    """CPU version of the Hénon-Heiles Hamiltonian."""
    T = 0.5 * (px**2 + py**2)
    V = 0.5 * (x**2 + y**2) + (x**2 * y - (y**3) / 3.0)
    return T + V

"""
Define potential energy function
"""
def V_henon_heiles(x, y):
    return 0.5 * (x**2 + y**2) + x**2 * y - (y**3) / 3.0

"""
Define Hamilton's equations of motion and variational equations
"""
@cuda.jit(device=True, inline=True)
def ode_henon_heiles_variational(t, Y, dYdt, params):
    """
    Calculates derivatives for the Hénon-Heiles system, its first two
    deviation vectors, and the Lagrangian Descriptor (LD) variable.
    Compatible with the generic adaptive variational solver kernel interface.

    State vector Y structure (size 13):
    [x, y, px, py, dx1, dy1, dpx1, dpy1, dx2, dy2, dpx2, dpy2, LD]

    Args:
        t (float): Current time (unused).
        Y (array): Current state vector.
        dYdt (array): Output array for derivatives.
        params (tuple): System parameters (unused for standard Hénon-Heiles).
    """
    # Unpack base state variables
    x = Y[0]
    y = Y[1]
    px = Y[2]
    py = Y[3]

    # Unpack deviation vector 1
    delta_x1 = Y[4]
    delta_y1 = Y[5]
    delta_px1 = Y[6]
    delta_py1 = Y[7]

    # Unpack deviation vector 2
    delta_x2 = Y[8]
    delta_y2 = Y[9]
    delta_px2 = Y[10]
    delta_py2 = Y[11]

    # --- Base Equations of Motion (Hénon-Heiles) ---
    dxdt = px
    dydt = py
    dpxdt = -x - 2.0 * x * y
    dpydt = -y - x * x + y * y

    # Store base derivatives
    dYdt[0] = dxdt
    dYdt[1] = dydt
    dYdt[2] = dpxdt
    dYdt[3] = dpydt

    # --- Variational Equations (Linearized dynamics) ---
    j31 = -1.0 - 2.0 * y
    j32 = -2.0 * x
    j41 = -2.0 * x
    j42 = -1.0 + 2.0 * y

    # Deviation Vector 1 Derivatives: d(delta)/dt = J * delta_1
    ddelta_x1dt = delta_px1
    ddelta_y1dt = delta_py1
    ddelta_px1dt = j31 * delta_x1 + j32 * delta_y1
    ddelta_py1dt = j41 * delta_x1 + j42 * delta_y1

    # Store deviation vector 1 derivatives
    dYdt[4] = ddelta_x1dt
    dYdt[5] = ddelta_y1dt
    dYdt[6] = ddelta_px1dt
    dYdt[7] = ddelta_py1dt

    # Deviation Vector 2 Derivatives: d(delta)/dt = J * delta_2
    ddelta_x2dt = delta_px2
    ddelta_y2dt = delta_py2
    ddelta_px2dt = j31 * delta_x2 + j32 * delta_y2
    ddelta_py2dt = j41 * delta_x2 + j42 * delta_y2

    # Store deviation vector 2 derivatives
    dYdt[8] = ddelta_x2dt
    dYdt[9] = ddelta_y2dt
    dYdt[10] = ddelta_px2dt
    dYdt[11] = ddelta_py2dt

    # --- Lagrangian Descriptor Derivative ---
    dLDdt = (math.sqrt(abs(dxdt)) +
             math.sqrt(abs(dydt)) +
             math.sqrt(abs(dpxdt)) +
             math.sqrt(abs(dpydt)))

    # Store LD derivative (index 12)
    dYdt[12] = dLDdt


For this example, initial conditions will be generated using an MC approach

In [3]:
"""
Generation of initial conditions
"""
def generate_ics_HH(num_ics, H_target, x0_section,
                           y_min, y_max, py_min, py_max,
                           num_vars, # Total vars including dev vecs + aux
                           seed=42, max_attempts_factor=100):
    """
    Generates initial conditions for Hénon-Heiles on a constant energy surface H
    at a fixed x=x0 Poincaré section. Samples y and py randomly, calculates px.
    Initializes 2 deviation vectors and 1 auxiliary variable (LD).

    Args:
        num_ics (int): Number of initial conditions desired.
        H_target (float): Target energy value.
        x0_section (float): Fixed x-coordinate for the section.
        y_min, y_max (float): Range for sampling y.
        py_min, py_max (float): Range for sampling py.
        num_vars (int): Total size of the state vector (should be 13 here).
        seed (int): Seed for the random number generator.
        max_attempts_factor (int): Factor to determine max attempts (num_ics * factor).

    Returns:
        np.ndarray: Array of initial conditions, shape (n_ics, num_vars).
                    Returns fewer than num_ics rows if max attempts reached.
    """
    if num_vars != 13:
        print(f"Warning: Expected num_vars=13 for HH+2dev+LD, got {num_vars}")
        # Adjust logic if state structure differs

    rng = default_rng(seed=seed)
    ics = np.zeros((num_ics, num_vars), dtype=np.float64)
    count = 0
    attempts = 0
    max_attempts = num_ics * max_attempts_factor

    print(f"Generating {num_ics} ICs for H={H_target} at x={x0_section}...")
    while count < num_ics and attempts < max_attempts:
        attempts += 1
        # Generate random y, py
        y_rand = rng.uniform(y_min, y_max)
        py_rand = rng.uniform(py_min, py_max)

        # Calculate potential energy at (x0, y_rand)
        V = V_henon_heiles(x0_section, y_rand)

        # Calculate px^2 = Delta = 2 * (H - T_y - V) = 2 * (H - 0.5*py^2 - V)
        delta_sq = 2.0 * (H_target - 0.5 * py_rand**2 - V)

        # Check if px is real (Delta > 0)
        if delta_sq > 1e-14: # Use small tolerance instead of exact zero
            px_calc = math.sqrt(delta_sq) # Taking positive root, as in MATLAB code

            # Store base variables
            ics[count, 0] = x0_section
            ics[count, 1] = y_rand
            ics[count, 2] = px_calc
            ics[count, 3] = py_rand

            # Initialize Deviation Vectors (using the simple fixed initialization)
            # Dev Vec 1 (Indices 4-7) -> [1e-5, 0, 0, 0]
            ics[count, 4] = 1e-5
            # ics[count, 5:8] = 0.0 # Already zero from np.zeros

            # Dev Vec 2 (Indices 8-11) -> [0, 1e-5, 0, 0]
            ics[count, 9] = 1e-5
            # ics[count, 8] = 0.0; ics[count, 10:12] = 0.0 # Already zero

            # Initialize LD aux var (Index 12)
            ics[count, 12] = 0.0

            count += 1 # Increment count ONLY for valid IC


        # else: Delta <= 0, invalid point, continue loop

    if count < num_ics:
        print(f"Warning: Only generated {count}/{num_ics} valid ICs after {max_attempts} attempts.")
        ics = ics[:count, :] # Return only the valid ones

    print(f"Finished generating {count} initial conditions.")
    return ics

In [4]:
"""
Define function to integrate trajectories on the GPU
"""
def integrate_trajectories_adaptive_variational(
    ics, solver_kernel, params_tuple, num_vars,
    t_final, tol, dt_initial, max_steps, renorm_interval
):
    """
    Integrates multiple trajectories (including base state and deviation vectors)
    in parallel on the GPU using a provided pre-compiled *adaptive-step variational*
    solver kernel with optional simple renormalization.

    Args:
        ics (np.ndarray): A 2D NumPy array of initial conditions, shape (n_ics, num_vars).
                          num_vars must match the kernel's expectation.
        solver_kernel: The Numba CUDA kernel function for the adaptive variational solver,
                       obtained by calling create_solver_kernel_adaptive_variational(...).
        params_tuple (tuple): A tuple containing the parameters required by the
                              ODE function.
        num_vars (int): The total number of variables in the ODE system (must match ics.shape[1]).
        t_final (float): The target final time for integration.
        tol (float): The absolute error tolerance for adaptive step control.
        dt_initial (float): The initial guess for the time step.
        max_steps (int): The maximum number of adaptive steps allowed per trajectory.
        renorm_interval (int): Renormalize deviation vectors every N *accepted* steps.
                               Set <= 0 to disable.

    Returns:
        np.ndarray: A 2D NumPy array containing the final states of the trajectories
                    after integration, shape (n_ics, num_vars). State is at the
                    time of the last accepted step.
    """
    n_ics = ics.shape[0] # Number of initial conditions
    if ics.shape[1] != num_vars:
        raise ValueError(f"Number of variables in ics ({ics.shape[1]}) does not match num_vars ({num_vars})")

    print(f"Starting adaptive variational integration for {n_ics} trajectories up to t={t_final}...")
    start_time = time.time()

    # --- Prepare GPU Data ---
    print("  Transferring initial conditions to GPU...")
    Y0_device = cuda.to_device(ics)

    print(f"  Allocating output array on GPU ({n_ics}x{num_vars})...")
    Y_out_device = cuda.device_array((n_ics, num_vars), dtype=np.float64)

    # --- Configure CUDA Launch ---
    blockdim = 256 # Or choose a suitable block dimension
    griddim = (n_ics + blockdim - 1) // blockdim
    print(f"  CUDA Launch Config: Grid={griddim}, Block={blockdim}")

    # --- Execute Adaptive Variational Kernel ---
    t0 = 0.0 # Initial time
    print(f"  Launching adaptive variational kernel (tol={tol}, dt_initial={dt_initial}, max_steps={max_steps}, renorm_interval={renorm_interval})...") # Added renorm_interval to printout
    # Call the PASSED-IN solver_kernel with the correct signature INCLUDING renorm_interval
    solver_kernel[griddim, blockdim](
        Y0_device,      # Initial states
        t0,             # Initial time
        t_final,        # Target final time
        params_tuple,   # Parameters tuple for the ODE function
        tol,            # Error tolerance
        dt_initial,     # Initial dt guess
        max_steps,      # Max allowed steps
        renorm_interval,# Renormalization interval
        Y_out_device    # Output array
    )
    cuda.synchronize() # Wait for the kernel to complete
    kernel_end_time = time.time()
    print(f"  Kernel execution finished in {kernel_end_time - start_time:.4f} seconds.")

    # --- Retrieve Results ---
    print("  Copying results back to host...")
    final_states = Y_out_device.copy_to_host()
    copy_end_time = time.time()
    print(f"  Data copy finished in {copy_end_time - kernel_end_time:.4f} seconds.")

    total_time = time.time() - start_time
    print(f"Total integration function time: {total_time:.4f} seconds.")
    print(f"Average integration time per IC: {total_time / n_ics:.6f} seconds.") # Still less meaningful

    return final_states

In [ ]:
"""
Define main function
"""
def main():

    # --- Simulation & System Parameters ---
    num_ics = 1000000 # number of initial conditions

    # Parameters for Hénon-Heiles IC generation
    H_target = 1.0 / 8.0 # Target energy value (example)
    x0_section = 0.0     # Poincaré section plane x=0
    y_min, y_max = -0.3, 0.5 # Sampling range for y (adjust as needed)
    py_min, py_max = -0.3, 0.3 # Sampling range for py (adjust as needed)

    # Define system structure
    NUM_BASE_VARS_HH = 4
    NUM_DEV_VECTORS_HH = 2
    NUM_AUX_VARS_HH = 1 # LD
    NUM_TOTAL_VARS_HH = NUM_BASE_VARS_HH * (1 + NUM_DEV_VECTORS_HH) + NUM_AUX_VARS_HH # 13
    params_tuple = () # No parameters for this ODE function

    # --- Integration Parameters ---
    t0 = 0.0
    t_final = 10000.0

    # --- Adaptive Integration Parameters ---
    abs_tolerance = 1e-8
    dt_initial_guess = 0.01
    max_steps_allowed = 2000000
    renorm_interval = 10 # Disable simple renormalization

    output = False # Set to True to save results
    if output is True:
        # --- Output Configuration ---
        output_dir = f"henon_heiles_results_E{H_target:.3f}_adaptive_sali" # Include Energy
        sali_dir = os.path.join(output_dir, "SALI")
        aux_info_dir = os.path.join(output_dir, "Aux_Info")
        ics_dir = os.path.join(output_dir, "ICS")
        os.makedirs(sali_dir, exist_ok=True); os.makedirs(aux_info_dir, exist_ok=True); os.makedirs(ics_dir, exist_ok=True)
        output_prefix = f"hh_adaptive_sali_n{num_ics}_E{H_target:.3f}" # Include Energy

        print("--- Hénon-Heiles Simulation (Adaptive Step, SALI, Energy Surface ICs) ---")
        print(f"Energy H={H_target}, Poincaré Section x0={x0_section}")
        print(f"Integration: t_final={t_final}, tol={abs_tolerance}, dt_init={dt_initial_guess}, max_steps={max_steps_allowed}")
        print(f"ICs: num_ics={num_ics}, y_range=[{y_min},{y_max}], py_range=[{py_min},{py_max}]")
        print(f"Output Dir: {output_dir}")

    # --- Generate Initial Conditions ---
    ics = generate_ics_HH(num_ics, H_target, x0_section,
                                 y_min, y_max, py_min, py_max,
                                 NUM_TOTAL_VARS_HH)
    if ics.shape[0] == 0:
        print("Error: No valid initial conditions generated. Exiting.")
        sys.exit(1)
    num_ics = ics.shape[0] # Update num_ics if filtering reduced it

    # --- Create the Adaptive Variational Solver Kernel ---
    print("\nCreating adaptive variational solver kernel...")
    adaptive_variational_solver = create_solver_kernel_adaptive_variational(
        ode_henon_heiles_variational,
        num_vars=NUM_TOTAL_VARS_HH,
        num_base_vars=NUM_BASE_VARS_HH,
        num_dev_vectors=NUM_DEV_VECTORS_HH
    )
    print("Kernel created.")

    # --- Integrate Trajectories ---
    print("\nIntegrating trajectories (adaptive)...")
    final_results = integrate_trajectories_adaptive_variational( # Use the correct wrapper
        ics=ics,
        solver_kernel=adaptive_variational_solver,
        params_tuple=params_tuple,
        num_vars=NUM_TOTAL_VARS_HH,
        t_final=t_final,
        tol=abs_tolerance,
        dt_initial=dt_initial_guess,
        max_steps=max_steps_allowed,
        renorm_interval=renorm_interval # Pass renorm interval
    )
    print("Integration finished.")

    # --- Energy Conservation Check ---
    print("\nChecking energy conservation...")
    initial_energies = np.array([H_henon_heiles_cpu(ic[0], ic[1], ic[2], ic[3]) for ic in ics])
    final_energies = np.array([H_henon_heiles_cpu(state[0], state[1], state[2], state[3]) for state in final_results])
    energy_diff = np.abs(final_energies - initial_energies)
    max_energy_diff = np.max(energy_diff) if energy_diff.size > 0 else np.nan
    print(f"  Target Energy H = {H_target}")
    print(f"  Max absolute energy deviation from initial: {max_energy_diff:.2e}")
    print(f"  Mean final energy: {np.mean(final_energies):.4f} +/- {np.std(final_energies):.2e}")


    # --- Compute SALI ---
    print("\nCalculating SALI...")
    sali_values = np.zeros(num_ics)
    calculation_errors = 0
    for i in range(num_ics):
        dev_vec1 = final_results[i, NUM_BASE_VARS_HH : NUM_BASE_VARS_HH * 2]
        dev_vec2 = final_results[i, NUM_BASE_VARS_HH * 2 : NUM_BASE_VARS_HH * 3]
        try:
            sali_values[i] = calc_SALI(dev_vec1, dev_vec2)
        except ValueError as e:
            print(f"Warning: Could not calculate SALI for IC {i}: {e}")
            sali_values[i] = np.nan
            calculation_errors += 1
    if calculation_errors > 0: print(f"Finished SALI calculation with {calculation_errors} errors.")
    else: print("SALI calculation finished.")
    print(f"SALI values: {sali_values[0:10]}")

    # Extract final LD values if needed (index 12)
    ld_values = final_results[:, NUM_BASE_VARS_HH * (1 + NUM_DEV_VECTORS_HH)]


    if output is True:
        # --- Save Results ---
        print("\nSaving results...")
        output_file_sali = os.path.join(sali_dir, f"{output_prefix}_sali.dat")
        output_file_ld = os.path.join(sali_dir, f"{output_prefix}_ld.dat") # Save LD too
        output_file_aux = os.path.join(aux_info_dir, f"{output_prefix}_aux.dat")
        output_file_ics = os.path.join(ics_dir, f"{output_prefix}_ics.dat")

        # Save SALI
        np.savetxt(output_file_sali, sali_values, delimiter=',', header='SALI', comments='')
        print(f"  Saved SALI values to: {output_file_sali}")
        # Save LD
        np.savetxt(output_file_ld, ld_values, delimiter=',', header='LD', comments='')
        print(f"  Saved LD values to: {output_file_ld}")

        # Save Auxiliary Info
        results_aux = np.array([[max_energy_diff, np.nan, np.nan, num_ics, H_target, x0_section, abs_tolerance, dt_initial_guess, max_steps_allowed]])
        np.savetxt(output_file_aux, results_aux, delimiter=',',
                   header='max_energy_diff,time_per_ic(NaN),total_time(NaN),num_ics,H,x0,tolerance,dt_initial,max_steps',
                   comments='')
        print(f"  Saved auxiliary info to: {output_file_aux}")

        # Save Main Initial Conditions (base variables only, or full state?)
        # Save full state for reproducibility
        np.savetxt(output_file_ics, ics, delimiter=',')
        print(f"  Saved initial conditions (full state) to: {output_file_ics}")

    print("\n--- Simulation Complete ---")


"""
Call main function
"""
if __name__ == "__main__":
    # Check for CUDA availability
    if not cuda.is_available():
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! CUDA is not available or not detected! !!!")
        print("!!! This script requires a CUDA-enabled GPU!!!")
        print("!!! and correctly installed CUDA drivers   !!!")
        print("!!! and Numba.                           !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        sys.exit(1) # Exit if no CUDA
    else:
        print(f"Found CUDA device: {cuda.get_current_device().name.decode()}")
        main() # Run the main function